# Bayesian optimization with fixed features
In some contexts a variable/feature needs to be fixed during optimization. However, we
can leverage previous measurements near the fixed variable value to potentially
jump-start optimization using observed model covariances established by the GP kernel
. In this example, we start with a number of random observations in 2D input space
and then proceed with BO at a fixed value for one of the variables. This notebook
uses the 2D Rosenbrock test function as an example.

In [ ]:
# set values if testing
import os

from xopt.generators.bayesian.visualize import plot_model_prediction
from xopt import Xopt, Evaluator
from xopt.generators.bayesian import UpperConfidenceBoundGenerator
from xopt.resources.test_functions.rosenbrock import (
    evaluate_rosenbrock,
    make_rosenbrock_vocs,
)

# Ignore all warnings
import warnings

warnings.filterwarnings("ignore")


SMOKE_TEST = os.environ.get("SMOKE_TEST")
NUM_MC_SAMPLES = 1 if SMOKE_TEST else 128
NUM_RESTARTS = 1 if SMOKE_TEST else 20

# make rosenbrock function vocs in 2D
vocs = make_rosenbrock_vocs(2)

# define a fixed value for the BO generator
fixed_features = {"x0": -1.0}
generator = UpperConfidenceBoundGenerator(vocs=vocs, fixed_features=fixed_features)
generator.numerical_optimizer.n_restarts = NUM_RESTARTS
generator.n_monte_carlo_samples = NUM_MC_SAMPLES

evaluator = Evaluator(function=evaluate_rosenbrock)

X = Xopt(generator=generator, evaluator=evaluator, vocs=vocs)
X

## Generate some initial random samples in 2D space

In [ ]:
X.random_evaluate(10)

## Run BO steps with fixed features

In [ ]:
for i in range(5):
    X.step()

In [ ]:
X.data

## Visualize model and evaluations
Note that for the BO samples, they all are on the line $x_0=-1$

In [ ]:
ax = plot_model_prediction(
    model=X.generator.model,
    vocs=X.vocs,
    data=X.data,
    show_samples=False,
    n_grid=100,
)
ax.plot(
    *X.data[["x0", "x1"]].to_numpy()[:10].T, "+C1", label="random samples", zorder=10
)
ax.plot(*X.data[["x0", "x1"]].to_numpy()[10:].T, "+C3", label="GP samples", zorder=10)
ax.axvline(-1.0, ls="--")
ax.legend();

## Run with fixed feature that is not in vocs
We can also run fixed features where the fixed variable is not listed in vocs, as
long as the generator data contains data corresponding to the fixed feature name. To
satisfy this requirements we add the data from the last optimization run.

In [ ]:
# make rosenbrock function vocs in 2-D but remove the `x0` name (set to a fixed
# feature in the next cell)
vocs = make_rosenbrock_vocs(2)
vocs.variables = {"x1": [-2.0, 2.0]}

In [ ]:
# define a fixed value for the BO generator
fixed_features = {"x0": -1.0}
generator = UpperConfidenceBoundGenerator(vocs=vocs, fixed_features=fixed_features)
generator.numerical_optimizer.n_restarts = NUM_RESTARTS
generator.n_monte_carlo_samples = NUM_MC_SAMPLES

evaluator = Evaluator(function=evaluate_rosenbrock)

X2 = Xopt(generator=generator, evaluator=evaluator, vocs=vocs)
X2.add_data(X.data)

In [ ]:
# run an optimization step
X2.step()

In [ ]:
X2.data